# Cross-Sections

This section demonstrates how to extract both horizontal and vertical cross-sections from an unstructured grid using UXarray, which allows the analysis and visualization across slices of grids. Cross-sections can be performed directly on a `ux.Grid` object or on a `ux.UxDataArray`.

## Horizontal Cross-Sections

Horizontal cross-sections extract slices at constant latitudes or longitudes on the horizontal plane.


In [ ]:
import cartopy.crs as ccrs
import geoviews as gv
import geoviews.feature as gf
import matplotlib.pyplot as plt

import uxarray as ux

projection = ccrs.Robinson()

In [ ]:
base_path = "../../test/meshfiles/ugrid/outCSne30/"
grid_path = base_path + "outCSne30.ug"
data_path = base_path + "outCSne30_vortex.nc"

uxds = ux.open_dataset(grid_path, data_path)
uxds["psi"].plot(
    cmap="inferno",
    periodic_elements="split",
    projection=projection,
    title="Global Plot",
)

### Constant Latitude

Horizontal cross-sections along constant latitude lines can be obtained by using the  ``.cross_section.constant_latitude(lat)`` method. The sliced grid will be made up of the faces that contain at least one edge that intersects with a line of constant latitude.


For example, we can obtain a cross-section at 0 degrees latitude by doing the following:

In [ ]:
lat = 0

Result of crossection call is a new ``UxDataArray``, we can directly plot the result to see the cross-section.  Below are two plots side-by-side using matplotlib.pyplot.subplots. The first plot will show the cross-section by selecting the grid faces that intersect the latitude line (sample=False), while the second will show an interpolated cross-section with 50 sample points (sample=True).

In [ ]:
# Create a figure with two subplots side-by-side
# sharey=True makes the y-axis the same for easier comparison
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(18, 6), sharey=True)

# --- Plot 1: Original Cross-Section (sample=False) ---

# Get the cross-section by selecting intersecting grid faces
uxda_constant_lat_1 = uxds["psi"].cross_section.constant_latitude(lat, sample=False)


# Plot the data values at the center of each intersecting face
face_lons_1 = uxda_constant_lat_1.uxgrid.face_lon.values
ax1.scatter(face_lons_1, uxda_constant_lat_1.values, c="blue", s=50, alpha=0.7)
ax1.set_xlabel("Longitude (face centers)")

ax1.set_ylabel("Data values")
ax1.set_title(f"Cross Section at {lat}° Latitude (sample=False)")
ax1.grid(True, alpha=0.3)

# --- Plot 2: Sampled Cross-Section (sample=True) ---

# Get the cross-section by sampling 50 points along the latitude line
uxda_constant_lat_2 = uxds["psi"].cross_section.constant_latitude(
    lat, sample=True, n_samples=10
)

# Plot the data on the second subplot (ax2)
# With sample=True, the result is always a structured line plot with 'lon' coordinates
uxda_constant_lat_2.plot(
    x="lon", ax=ax2, marker="o", linewidth=2, markersize=4, color="red"
)
ax2.set_xlabel("Longitude")
ax2.set_title(f"Cross Section at {lat}° Latitude (sample=True, n_samples=10)")
ax2.grid(True, alpha=0.3)


# --- Display the final figure ---
plt.tight_layout()
plt.show()

You can also perform operations on the cross-section, such as taking the mean.

In [ ]:
print(f"Global Mean: {uxds['psi'].data.mean()}")
print(f"Mean at {lat} degrees lat: {uxda_constant_lat_2.data.mean()}")

### Constant Longitude


Horizontal cross-sections along constant longitude lines can be obtained using the ``.cross_section.constant_longitude(lon)`` method. The sliced grid will be made up of the faces that contain at least one edge that intersects with a line of constant longitude.


In [ ]:
lon = 90

uxda_constant_lon = uxds["psi"].cross_section.constant_longitude(lon)

In [ ]:
(
    uxda_constant_lon.plot.polygons(
        rasterize=False,
        backend="bokeh",
        cmap="inferno",
        projection=projection,
        global_extent=True,
        coastline=True,
        title=f"Cross Section at {lon} degrees longitude",
        periodic_elements="split",
    )
    * gf.grid(projection=projection)
)

### Constant Latitude Interval

Horizontal cross-sections between two lines of latitudes can be obtained using the ``.cross_section.constant_latitude_interval(lats)`` method. The sliced grid will contain faces that are strictly between the latitude interval.

In [ ]:
lats = [-20, 20]

uxda_constant_lat_interval = uxds["psi"].cross_section.constant_latitude_interval(lats)

In [ ]:
(
    uxda_constant_lat_interval.plot.polygons(
        rasterize=False,
        backend="bokeh",
        cmap="inferno",
        projection=projection,
        global_extent=True,
        coastline=True,
        title=f"Cross Section between {lats[0]} and {lats[1]} degrees latitude",
        periodic_elements="split",
    )
    * gf.grid(projection=projection)
)

### Constant Longitude Interval

Horizontal cross-sections between two lines of longitude can be obtained using the ``.cross_section.constant_longitude_interval(lons)`` method. The sliced grid will contain faces that are strictly between the longitude interval.


In [ ]:
lons = [-25, 25]

uxda_constant_lon_interval = uxds["psi"].cross_section.constant_longitude_interval(lons)

In [ ]:
(
    uxda_constant_lon_interval.plot.polygons(
        rasterize=False,
        backend="bokeh",
        cmap="inferno",
        projection=projection,
        global_extent=True,
        coastline=True,
        title=f"Cross Section between {lons[0]} and {lons[1]} degrees longitude",
        periodic_elements="split",
    )
    * gf.grid(projection=projection)
)

## Vertical Cross-Sections

Vertical cross-sections work on 3D data (with vertical levels like atmospheric pressure levels, ocean depths, etc.) and extract slices along constant latitudes or longitudes while preserving ALL vertical levels. This creates a 2D vertical transect showing how data varies spatially and vertically.

Cross-sections create **regular sampling** along the specified line, making them perfect for visualization and analysis.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create 3D data with vertical levels for demonstration
pressure_levels = np.array([1000, 850, 700, 500, 300])  # hPa
psi_3d = uxds["psi"].expand_dims(pressure=pressure_levels)

print(f"Original 2D data shape: {uxds['psi'].shape}")
print(f"3D data shape: {psi_3d.shape}")
print(f"3D data dimensions: {psi_3d.dims}")

In [ ]:
# Extract cross-section at constant latitude
lat = 0.0
cross_section = psi_3d.cross_section.constant_latitude(
    lat=lat, n_samples=50, lon_range=(-180, 180)
)

print(f"Cross-section at {lat}° latitude:")
print(f"  Shape: {cross_section.shape}")
print(f"  Dimensions: {cross_section.dims}")
print(f"  Type: {type(cross_section)}")
print(f"  Coordinates: {list(cross_section.coords.keys())}")
print("  → Creates regular sampling perfect for visualization")

In [ ]:
# Create vertical cross-section visualization
fig, ax = plt.subplots(figsize=(12, 6))

# Plot the cross-section as a 2D field
cross_section.plot(x="lon", y="pressure", ax=ax, cmap="viridis")
ax.set_xlabel("Longitude (°)")
ax.set_ylabel("Pressure (hPa)")
ax.set_title(f"Vertical Cross-Section at {lat}° Latitude")
ax.invert_yaxis()  # Pressure decreases upward
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Longitude Cross-Sections

Similarly, you can extract cross-sections at constant longitudes (meridional transects):

In [ ]:
# Extract longitude cross-section (meridional transect)
lon = 0.0
lon_section = psi_3d.cross_section.constant_longitude(
    lon=lon, n_samples=30, lat_range=(-90, 90)
)

print(f"Longitude cross-section shape: {lon_section.shape}")
print(f"Longitude cross-section dims: {lon_section.dims}")
print(f"Coordinates: {list(lon_section.coords.keys())}")

In [ ]:
# Visualize the longitude cross-section
fig, ax = plt.subplots(figsize=(10, 6))

lon_section.plot(x="lat", y="pressure", ax=ax, cmap="plasma")
ax.set_xlabel("Latitude (°)")
ax.set_ylabel("Pressure (hPa)")
ax.set_title(f"Meridional Cross-Section at {lon}° Longitude")
ax.invert_yaxis()  # Pressure decreases upward
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Arbitrary Great Circle Arc (GCA)


```{warning}
Arbitrary great circle arc cross sections are not yet implemented.
```